# Global Tweet Sentiment Analysis - Mood of the World

# Install Required Packages

In [ ]:
!pip install tweepy textblob plotly pandas numpy wordcloud matplotlib seaborn python-dotenv --quiet
!python -m textblob.download_corpora

# Import Libraries

In [ ]:
import tweepy
import pandas as pd
import numpy as np
from textblob import TextBlob
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
from datetime import datetime, timedelta
import re
import json
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
pyo.init_notebook_mode(connected=True)

# Twitter API Configuration
## Note : get API from https://developer.twitter.com/


In [ ]:
class TwitterConfig:
    def __init__(self):
        # Replace with your actual API credentials
        self.API_KEY = "YOUR_API_KEY"
        self.API_SECRET = "YOUR_API_SECRET"
        self.ACCESS_TOKEN = "YOUR_ACCESS_TOKEN"
        self.ACCESS_TOKEN_SECRET = "YOUR_ACCESS_TOKEN_SECRET"
        self.BEARER_TOKEN = "YOUR_BEARER_TOKEN"

    def get_api_v1(self):
        """Get Twitter API v1.1 client"""
        auth = tweepy.OAuthHandler(self.API_KEY, self.API_SECRET)
        auth.set_access_token(self.ACCESS_TOKEN, self.ACCESS_TOKEN_SECRET)
        return tweepy.API(auth, wait_on_rate_limit=True)

    def get_api_v2(self):
        """Get Twitter API v2 client"""
        return tweepy.Client(
            bearer_token=self.BEARER_TOKEN,
            consumer_key=self.API_KEY,
            consumer_secret=self.API_SECRET,
            access_token=self.ACCESS_TOKEN,
            access_token_secret=self.ACCESS_TOKEN_SECRET,
            wait_on_rate_limit=True
        )
config = TwitterConfig()

# Data Collection

In [ ]:
class GlobalTweetCollector:
    def __init__(self, api_v1, api_v2):
        self.api_v1 = api_v1
        self.api_v2 = api_v2
        self.tweets_data = []

    def clean_tweet(self, text):
        """Clean tweet text for better sentiment analysis"""
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
        text = re.sub(r'@\w+|#\w+', '', text)
        text = ' '.join(text.split())
        return text.strip()

    def get_sentiment(self, text):
        """Analyze sentiment using TextBlob"""
        try:
            blob = TextBlob(text)
            polarity = blob.sentiment.polarity
            subjectivity = blob.sentiment.subjectivity
            if polarity > 0.1:
                sentiment = 'Positive'
            elif polarity < -0.1:
                sentiment = 'Negative'
            else:
                sentiment = 'Neutral'

            return {
                'sentiment': sentiment,
                'polarity': polarity,
                'subjectivity': subjectivity
            }
        except:
            return {
                'sentiment': 'Neutral',
                'polarity': 0.0,
                'subjectivity': 0.0
            }

    def collect_global_tweets(self, keywords=None, count=1000):
        """Collect tweets from around the world"""
        if keywords is None:
            keywords = [
                "good morning", "how are you", "feeling", "mood", "happy",
                "sad", "excited", "worried", "grateful", "blessed",
                "monday", "tuesday", "wednesday", "thursday", "friday",
                "weekend", "today", "life", "work", "family"
            ]

        print(f"Starting global tweet collection...")

        for keyword in keywords:
            try:
                print(f"Searching for: '{keyword}'")
                tweets = tweepy.Paginator(
                    self.api_v2.search_recent_tweets,
                    query=f"{keyword} -is:retweet lang:en",
                    tweet_fields=['created_at', 'author_id', 'public_metrics', 'geo'],
                    max_results=100,
                    limit=10
                ).flatten(limit=count // len(keywords))

                for tweet in tweets:
                    if tweet.text:
                        cleaned_text = self.clean_tweet(tweet.text)
                        if len(cleaned_text) > 10:
                            sentiment_data = self.get_sentiment(cleaned_text)

                            tweet_data = {
                                'id': tweet.id,
                                'text': cleaned_text,
                                'original_text': tweet.text,
                                'created_at': tweet.created_at,
                                'author_id': tweet.author_id,
                                'keyword': keyword,
                                'sentiment': sentiment_data['sentiment'],
                                'polarity': sentiment_data['polarity'],
                                'subjectivity': sentiment_data['subjectivity'],
                                'retweet_count': tweet.public_metrics['retweet_count'] if tweet.public_metrics else 0,
                                'like_count': tweet.public_metrics['like_count'] if tweet.public_metrics else 0,
                            }

                            self.tweets_data.append(tweet_data)

            except Exception as e:
                print(f"Error collecting tweets for '{keyword}': {str(e)}")
                continue

        print(f"Collected {len(self.tweets_data)} tweets total")
        return pd.DataFrame(self.tweets_data)

# Initialize APIs and Collect Data or Sample Data for Demonstration

In [ ]:
try:
    api_v1 = config.get_api_v1()
    api_v2 = config.get_api_v2()

    print("Testing Twitter API connection...")
    me = api_v1.verify_credentials()
    if me:
        print(f"Connected as: @{me.screen_name}")
    else:
        print("API connection failed.")
    collector = GlobalTweetCollector(api_v1, api_v2)
    df_tweets = collector.collect_global_tweets(count=2000)

    if len(df_tweets) > 0:
        print(f"Dataset shape: {df_tweets.shape}")
        print("\n Dataset info:")
        print(df_tweets.info())
        print(f"\n Sentiment distribution:")
        print(df_tweets['sentiment'].value_counts())
    else:
        print("No tweets collected. Check your API credentials.")

except Exception as e:
    print(f"Error: {str(e)}")

    print("\n Creating sample data for demonstration...")
    sample_data = {
        'text': [
            'Having a great day today!',
            'Feeling a bit down lately',
            'Just another normal Monday',
            'So excited for the weekend!',
            'Work is really stressing me out',
            'Grateful for my family and friends',
            'Weather is perfect today',
            'This week has been challenging',
            'Looking forward to vacation',
            'Love spending time with loved ones'
        ] * 50,
        'sentiment': ['Positive', 'Negative', 'Neutral', 'Positive', 'Negative'] * 100,
        'polarity': np.random.normal(0, 0.3, 500),
        'subjectivity': np.random.uniform(0, 1, 500),
        'created_at': pd.date_range(start='2024-01-01', periods=500, freq='H'),
        'keyword': np.random.choice(['mood', 'feeling', 'today', 'happy', 'work'], 500)
    }
    df_tweets = pd.DataFrame(sample_data)
    print(f"Sample dataset created with {len(df_tweets)} tweets")

# Data Analysis

In [ ]:
class SentimentAnalyzer:
    def __init__(self, df):
        self.df = df.copy()
        self.prepare_data()

    def prepare_data(self):
        """Prepare data for analysis"""
        if 'created_at' in self.df.columns:
            self.df['created_at'] = pd.to_datetime(self.df['created_at'])
            self.df['hour'] = self.df['created_at'].dt.hour
            self.df['day_of_week'] = self.df['created_at'].dt.day_name()
            self.df['date'] = self.df['created_at'].dt.date

    def get_sentiment_summary(self):
        """Get overall sentiment statistics"""
        total_tweets = len(self.df)
        sentiment_counts = self.df['sentiment'].value_counts()

        summary = {
            'total_tweets': total_tweets,
            'positive_pct': (sentiment_counts.get('Positive', 0) / total_tweets) * 100,
            'negative_pct': (sentiment_counts.get('Negative', 0) / total_tweets) * 100,
            'neutral_pct': (sentiment_counts.get('Neutral', 0) / total_tweets) * 100,
            'avg_polarity': self.df['polarity'].mean(),
            'avg_subjectivity': self.df['subjectivity'].mean()
        }

        return summary

    def get_hourly_sentiment(self):
        """Analyze sentiment by hour of day"""
        hourly_sentiment = self.df.groupby(['hour', 'sentiment']).size().unstack(fill_value=0)
        hourly_polarity = self.df.groupby('hour')['polarity'].mean()

        return hourly_sentiment, hourly_polarity

    def get_daily_sentiment(self):
        """Analyze sentiment by day of week"""
        daily_sentiment = self.df.groupby(['day_of_week', 'sentiment']).size().unstack(fill_value=0)
        daily_polarity = self.df.groupby('day_of_week')['polarity'].mean()

        return daily_sentiment, daily_polarity

    def get_keyword_sentiment(self):
        """Analyze sentiment by keyword"""
        if 'keyword' in self.df.columns:
            keyword_sentiment = self.df.groupby(['keyword', 'sentiment']).size().unstack(fill_value=0)
            keyword_polarity = self.df.groupby('keyword')['polarity'].mean().sort_values(ascending=False)
            return keyword_sentiment, keyword_polarity
        return None, None

analyzer = SentimentAnalyzer(df_tweets)
summary = analyzer.get_sentiment_summary()

print("Mood of the World SUMMARY")
print("=" * 40)
print(f"Total Tweets Analyzed: {summary['total_tweets']:,}")
print(f"Positive Sentiment: {summary['positive_pct']:.1f}%")
print(f"Negative Sentiment: {summary['negative_pct']:.1f}%")
print(f"Neutral Sentiment: {summary['neutral_pct']:.1f}%")
print(f"Average Polarity: {summary['avg_polarity']:.3f}")
print(f"Average Subjectivity: {summary['avg_subjectivity']:.3f}")